# Introduction to atomman: Defect creation and analysis

__Lucas M. Hale__, [lucas.hale@nist.gov](mailto:lucas.hale@nist.gov?Subject=ipr-demo), _Materials Science and Engineering Division, NIST_.
    
[Disclaimers](http://www.nist.gov/public_affairs/disclaimer.cfm) 

## 1. Introduction<a id='section1'></a>

One of the focuses of the atomman package is to provide tools for generating and analyzing atomic systems containing defects.  Many of these tools are more complex than the basic tools outlined in the previous section. This Notebook provides an overview of the various defect capabilities in atomman.

**Library Imports**

In [1]:
# Standard Python libraries
import datetime

# http://www.numpy.org/
import numpy as np

# https://github.com/usnistgov/atomman
import atomman as am
import atomman.unitconvert as uc

# https://matplotlib.org/
import matplotlib.pyplot as plt
%matplotlib inline

# Show atomman version
print('atomman version =', am.__version__)

# Show date of Notebook execution
print('Notebook executed on', datetime.date.today())

atomman version = 1.4.8
Notebook executed on 2023-01-27


## 2. Point defects<a id='section2'></a>

The atomman.defect.point() function inserts a point defect into a System and helps with tracking the defect by (1) moving any new/modified atoms to the end of the Atoms list and (2) adding per-atom property old_id that retains each atom's original index.

See [4.1. Point defect generation Jupyter Notebook](4.1. Point defect generation.ipynb) for more examples.

- __system__ (*atomman.System*) the system to add the defect to.
- __ptd_type__ (*str*) indicates which type of point defect to add.
    - ='v' -- vacancy
    - ='s' -- substitutional
    - ='i' -- positional interstitial
    - ='db' -- dumbbell interstitial
- __atype__ (*int, optional*) atom type for defect atom ('i', 's', 'db' styles).
- __pos__ (*array-like object, optional*) position for adding the defect atom (all styles).
- __ptd_id__ (*int, optional*) atom id where defect is added.  Alternative to using pos ('v', 's', 'db' styles).
- __db_vect__ (*array-like object, optional*) vector associated with the dumbbell interstitial ('db' style).
- __scale__ (*bool, optional*) indicates if pos and db_vect are absolute (False) or box-relative (True). Default is False.
- __atol__ (*float, optional*) absolute tolerance for position-based searching. Default is 1e-3 angstroms.

In [2]:
# Define 2x2x2 bcc demonstration supercell
a = uc.set_in_units(2.8665, 'angstrom')
box = am.Box(a=a, b=a, c=a)
atoms = am.Atoms(atype=1, pos=[[0.0,0.0,0.0], [0.5,0.5,0.5]])
ucell = am.System(atoms=atoms, box=box, scale=True)
system = ucell.supersize(2,2,2)
print(system.atoms_df())

    atype   pos[0]   pos[1]   pos[2]
0       1  0.00000  0.00000  0.00000
1       1  1.43325  1.43325  1.43325
2       1  2.86650  0.00000  0.00000
3       1  4.29975  1.43325  1.43325
4       1  0.00000  2.86650  0.00000
5       1  1.43325  4.29975  1.43325
6       1  2.86650  2.86650  0.00000
7       1  4.29975  4.29975  1.43325
8       1  0.00000  0.00000  2.86650
9       1  1.43325  1.43325  4.29975
10      1  2.86650  0.00000  2.86650
11      1  4.29975  1.43325  4.29975
12      1  0.00000  2.86650  2.86650
13      1  1.43325  4.29975  4.29975
14      1  2.86650  2.86650  2.86650
15      1  4.29975  4.29975  4.29975


In [3]:
# Change atom 5 into <111> dumbbell interstitial pair
defect_system = am.defect.point(system, 'db', ptd_id=5, db_vect=[0.3, 0.3, 0.3])
print(defect_system.atoms_df())

    atype   pos[0]   pos[1]   pos[2]  old_id
0       1  0.00000  0.00000  0.00000       0
1       1  1.43325  1.43325  1.43325       1
2       1  2.86650  0.00000  0.00000       2
3       1  4.29975  1.43325  1.43325       3
4       1  0.00000  2.86650  0.00000       4
5       1  2.86650  2.86650  0.00000       6
6       1  4.29975  4.29975  1.43325       7
7       1  0.00000  0.00000  2.86650       8
8       1  1.43325  1.43325  4.29975       9
9       1  2.86650  0.00000  2.86650      10
10      1  4.29975  1.43325  4.29975      11
11      1  0.00000  2.86650  2.86650      12
12      1  1.43325  4.29975  4.29975      13
13      1  2.86650  2.86650  2.86650      14
14      1  4.29975  4.29975  4.29975      15
15      1  1.13325  3.99975  1.13325       5
16      1  1.73325  4.59975  1.73325      16


## 3. Free surfaces<a id='section3'></a>

Atomic configurations with a specific crystalline free surface can be generated with the FreeSurface class.  

See the [4.2. Free surface generator Jupyter Notebook](4.2. Free surface generator.ipynb) for the underlying theory and examples of the class being used.

## 4. Stacking faults<a id='section4'></a>

The atomman.defect.StackingFault class provides methods for generating atomic configurations of generalized stacking faults that can be used to compute stacking fault energies.  You can generate either a single configuration, or a 1D array of configurations for a slip plane and direction, or a 2D map of configurations exploring the full gamma surface.

See the [4.3. Stacking fault generator Jupyter Notebook](4.3. Stacking fault generator.ipynb) for a description of the class and examples.

The atomman.defect.GammaSurface class provides convenient methods for plotting and interpolating generalized stacking fault ($\gamma$ surface) data.

See the [4.5. Gamma surface plotting Jupyter Notebook](4.5. Gamma surface plotting.ipynb) for a description of the class and examples.

## 5. Dislocations<a id='section5'></a>

*Updated version 1.2.5:* Ability to solve isotropic solutions added.

*Added version 1.3.4:* Dislocation class

The atomman.defect.solve_volterra_dislocation() function generates an elasticity solution for a perfectly straight dislocation. The function returns either a Stroh object or a IsotropicVolterraDislocation object which has methods for obtaining position-based displacements and stress states, as well as energy-related parameters.  

The atomman.defect.Dislocation class provides a convenient tool for constructing atomic configurations of dislocations based on the Volterra solutions. 

See [4.4. Dislocation solution and generator Jupyter Notebook](4.4. Dislocation solution and generator.ipynb) for more details about the Volterra solvers and the dislocation configuration generation methods.

There are also a number of analysis tools for characterizing dislocations and other crystalline defects.

- **atomman.defect.slip_vector()** calculates the slip vector for all atoms in a system.  This is a good tool for characterizing plastic deformation within a system that occurs between two points in time. 
- **atomman.defect.disregistry()** characterizes a dislocation's planar spreading by calculating the relative displacement of atoms between the atomic planes above and below the dislocation's slip plane. 
- **atomman.defect.DifferentialDisplacement** allows for dislocation cores to be characterized using the difference in the displacements between all pairs of neighbor atoms relative to a perfect crystal configuration.

See the [4.6. Dislocation analysis tools Jupyter Notebook](4.6. Dislocation analysis tools.ipynb) for the underlying theory and examples of the tools being used.

The atomman.defect.SDVPN class allows for Semidiscrete variational Peierls-Nabarro dislocation models to be constructed based on supplied generalized stacking fault energy values.

See the [4.7. Semidiscrete variational Peierls-Nabarro model Jupyter Notebook](4.7. Semidiscrete variational Peierls-Nabarro model.ipynb) for a description of the method and examples. 

## 6. Strain class <a id='section6'></a>

*Added version 1.3.7*

The atomman.defect.Strain class provides a means of computing atomistic strain properties on a per-atom basis.  In performing the calculation, the reference "zero strain" configuration can be specified either as a System object or by specifying ideal lattice vectors on a per-atom basis.

See the [4.8. Strain class Juptyer Notebook](4.8. Strain class.ipynb) for a description of the class and examples. 